# AWS Lambda - S3


## Setup



If you have chosen to use an rc file, source it as ```source <your-aws-credentials-rc-file>```, e.g.

In [ ]:
. ~/.aws/credentials.rc

In [209]:
cleanup_functions() {
    echo "-- Functions before:"
    aws lambda list-functions | grep FunctionName

    FNS=$(aws lambda list-functions | grep FunctionName |sed -e 's/.*: //'  -e 's/"//g' -e 's/,//')
    for FN in $FNS; do aws lambda delete-function --function-name $FN; done
    echo "-- Functions after cleanup:"
    aws lambda list-functions | grep FunctionName
}

In [ ]:
cd ~/src/git/ServerlessLabs/ServerlessWorkshop/AWS-S3-Lambda

In [210]:
cleanup_functions

-- Functions before:
            "FunctionName": "chalice-app-dev",
            "FunctionName": "scheduled-app-dev-periodic_task",
            "FunctionName": "s3-app-dev-handler",
-- Functions after cleanup:


: 1

In [ ]:
chalice new-project s3-app
cd s3-app

In [211]:
aws lambda list-functions

{
    "Functions": []
}


# From Lambda

Remove old bucket if present to avoid possible event configuration conflict

In [213]:
aws s3 rb --force s3://mjbright-uploads

delete: s3://mjbright-uploads/testfile
remove_bucket: mjbright-uploads


In [214]:
aws s3 mb s3://mjbright-uploads
aws s3 ls

make_bucket: mjbright-uploads
2019-01-25 21:41:22 mjbright-static-site
2019-01-26 07:00:20 mjbright-uploads


In [221]:
BUCKET_NAME='mjbright-uploads'

cat > app.py <<EOF
from chalice import Chalice
#import sys

app = Chalice(app_name="s3-uploads")

# Whenever an object is uploaded to 'mybucket'
# this lambda function will be invoked.

@app.on_s3_event(bucket='$BUCKET_NAME')
def handler(event):
    # Output goes to CloudWatch logs - viewable via 'chalice logs'
    print("Object uploaded for bucket: %s, key: %s"
          % (event.bucket, event.key))
    #sys.exit(0)
EOF

In [222]:
pwd; cat app.py

/home/user1/src/git/ServerlessLabs/ServerlessWorkshop/AWS-S3-Lambda/s3-app
from chalice import Chalice
#import sys

app = Chalice(app_name="s3-uploads")

# Whenever an object is uploaded to 'mybucket'
# this lambda function will be invoked.

@app.on_s3_event(bucket='mjbright-uploads')
def handler(event):
    # Output goes to CloudWatch logs - viewable via 'chalice logs'
    print("Object uploaded for bucket: %s, key: %s"
          % (event.bucket, event.key))
    #sys.exit(0)


Deploy a local chalice server in another window using ```chalice local```, then upload a file to our bucket

In [223]:
cat > requirements.txt <<EOF
boto3==1.3.1
EOF

In [218]:
# Only works when deployed - local server has no S3 access
chalice deploy

/usr/share/python-wheels/requests-2.18.4-py2.py3-none-any.whl/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
Creating deployment package.
Updating policy for IAM role: s3-app-dev
Updating lambda function: s3-app-dev-handler
Configuring S3 events in bucket mjbright-uploads to function s3-app-dev-handler
Resources deployed:
  - Lambda ARN: arn:aws:lambda:us-west-1:568285458700:function:s3-app-dev-handler


In [219]:
aws s3 ls s3://mjbright-uploads

In [224]:
aws s3 cp app.py s3://mjbright-uploads/testfile

upload: ./app.py to s3://mjbright-uploads/testfile                


In [225]:
aws s3 ls s3://mjbright-uploads

2019-01-26 07:02:59        406 testfile


In [227]:
chalice logs --help

Usage: chalice logs [OPTIONS]

Options:
  --num-entries INTEGER           Max number of log entries to show.
  --include-lambda-messages / --no-include-lambda-messages
                                  Controls whether or not lambda log messages
                                  are included.
  --stage TEXT
  -n, --name TEXT                 The name of the lambda function to retrieve
                                  logs from.
  --profile TEXT                  The profile to use for fetching logs.
  --help                          Show this message and exit.


In [231]:
chalice logs --include-lambda-messages

In [220]:
pwd

/home/user1/src/git/ServerlessLabs/ServerlessWorkshop/AWS-S3-Lambda/s3-app


In [164]:
cat > app.py <<EOF

from chalice import Chalice
from chalice import NotFoundError
import json
import pprint
pp = pprint.PrettyPrinter(indent=4)

app = Chalice(app_name='s3')

import json
import boto3
from botocore.exceptions import ClientError

S3 = boto3.client('s3', region_name='us-west-1')
BUCKET = 'mjbright-uploads'

@app.route('/text/{key}', methods=['GET', 'PUT'])
def s3text(key):
    request = app.current_request
    
    # Log request information to CloudWatch
    print("s3text: -----------------------------------------------------------------"); pp.pprint(request.to_dict())
    
    # PUT a new S3 object:
    if request.method == 'PUT':
        print("s3text: PUT s3://{}/{}".format(BUCKET,key))
        #S3.put_object(Bucket=BUCKET, Key=key, Body=str(request.json_body))
        jb = request.json_body
        if jb == null:
            print("jb == null")
        else:
            print( str(jb))
        S3.put_object(Bucket=BUCKET, Key=key, Body=json.dumps(request.json_body))

    # GET an existing S3 object, if it exists:
    elif request.method == 'GET':
        try:
            print("s3text: GET s3://{}/{}".format(BUCKET,key))
            response = S3.get_object(Bucket=BUCKET, Key=key)
            return str(response['Body'].read())
        except ClientError as e:
            raise NotFoundError(key)

@app.route('/json/{key}', methods=['GET', 'PUT'])
def s3json(key):
    request = app.current_request
    
    # Log request information to CloudWatch
    print("s3json: -----------------------------------------------------------------"); pp.pprint(request.to_dict())
    print("json_body: -----------------------------------------------------------------"); pp.pprint(request.json_body)
    print("json_body: -----------------------------------------------------------------"); print(request.json_body)
    print("len(json_body): ", len(request.json_body))
    
    # PUT a new S3 object:
    if request.method == 'PUT':
        print("s3json: PUT s3://{}/{}".format(BUCKET,key))
        S3.put_object(Bucket=BUCKET, Key=key,
                      Body=json.dumps(request.json_body))

    # GET an existing S3 object, if it exists:
    elif request.method == 'GET':
        try:
            print("s3json: GET s3://{}/{}".format(BUCKET,key))
            response = S3.get_object(Bucket=BUCKET, Key=key)
            return json.loads(response['Body'].read())
        except ClientError as e:
            raise NotFoundError(key)

EOF

## NOTE: local test

This will not work because local server hasn't S3 credentials, but at least we might learn something from error messages

In [165]:
#echo "{ 'hello': 'world'; }" | http PUT :8000/json/test
echo "{ 'a': 'b'; };" > test
echo "{ 'a': 'b' }" > test
cat test

#cat test | http PUT :8000/json/test
cat test | http PUT :8000/text/test

{ 'a': 'b' }
/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
HTTP/1.1 400 Bad Request
Content-Length: 74
Content-Type: application/json
Date: Sat, 26 Jan 2019 01:51:01 GMT
Server: BaseHTTP/0.6 Python/3.6.7

{
    "Code": "BadRequestError",
    "Message": "BadRequestError: Error Parsing JSON"
}



In [116]:
echo hello world | http PUT :8000/text/test

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
HTTP/1.1 500 Internal Server Error
Content-Length: 77
Content-Type: application/json
Date: Sat, 26 Jan 2019 01:10:37 GMT
Server: BaseHTTP/0.6 Python/3.6.7

{
    "Code": "InternalServerError",
    "Message": "An internal server error occurred."
}



In [102]:
http :8000/json/test; http :8000/text/test;

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
HTTP/1.1 500 Internal Server Error
Content-Length: 77
Content-Type: application/json
Date: Sat, 26 Jan 2019 01:03:06 GMT
Server: BaseHTTP/0.6 Python/3.6.7

{
    "Code": "InternalServerError",
    "Message": "An internal server error occurred."
}

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
HTTP/1.1 500 Internal Server Error
Content-Length: 77
Content-Type: application/json
Date: Sat, 26 Jan 2019 01:03:06 GMT
Server: BaseHTTP/0.6 Python/3.6.7

{
    "Code": "InternalServerError",
    "Message": "An internal server error occurred."
}



In [103]:
chalice deploy

/usr/share/python-wheels/requests-2.18.4-py2.py3-none-any.whl/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
Creating deployment package.
Updating policy for IAM role: chalice-app-dev
Updating lambda function: chalice-app-dev
Updating rest API
Resources deployed:
  - Lambda ARN: arn:aws:lambda:us-west-1:568285458700:function:chalice-app-dev
  - Rest API URL: https://jlmq3ygdu1.execute-api.us-west-1.amazonaws.com/api/


In [123]:
echo "{ 'a': 'b' }" > test
aws s3 cp test s3://mjbright-uploads/test

upload: ./test to s3://mjbright-uploads/test                      


In [105]:
aws s3 ls s3://mjbright-uploads/

                           PRE objects/
2019-01-26 00:19:14        900 app.py
2019-01-26 00:18:18        221 hosts
2019-01-26 01:03:32         13 test


In [124]:
cat test

{ 'a': 'b' }


In [108]:
http https://jlmq3ygdu1.execute-api.us-west-1.amazonaws.com/api/text/test
http https://jlmq3ygdu1.execute-api.us-west-1.amazonaws.com/api/json/test

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
HTTP/1.1 200 OK
Connection: keep-alive
Content-Length: 17
Content-Type: application/json
Date: Sat, 26 Jan 2019 01:04:01 GMT
Via: 1.1 49c80a47c1441dd194a8337982f1cd7e.cloudfront.net (CloudFront)
X-Amz-Cf-Id: x5l6UO5ZAWQVuG_3tPQWMiZHcMuAD2DrC2PGoGYLvUlkS2Y6zaXXBg==
X-Amzn-Trace-Id: Root=1-5c4bb201-b56313b28f6fdc1ce3892232;Sampled=0
X-Cache: Miss from cloudfront
x-amz-apigw-id: UFjAOGudyK4FSUw=
x-amzn-RequestId: 447e02ca-2106-11e9-86ac-6596e4f7f276

b"{ 'a': 'b' }\n"

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
HTTP/1.1 500 Internal Server Error
Connection: keep-alive
Content-Length: 77
Content-Type: application/json
Date: Sat, 26 Jan 2019 01:04:02 GMT
Via: 1.1 8008015354

In [110]:
chalice logs --include-lambda-messages | grep s3

2019-01-26 00:22:26.050000 c313db s3objects: <<chalice.app.Request object at 0x7ffa1f2c6940>>
2019-01-26 00:24:34.082000 a295f6 s3objects: <<chalice.app.Request object at 0x7fa8a7cd6940>>
2019-01-26 00:27:26.818000 c569b0 s3 - ERROR - Internal Error for <function s3objects at 0x7f22ac1fc730>
2019-01-26 00:27:26.818000 c569b0 File "/var/task/app.py", line 20, in s3objects
2019-01-26 00:32:18.695000 b6528d s3objects: <{"query_params": null, "headers": {"accept": "*/*", "accept-encoding": "gzip, deflate", "cloudfront-forwarded-proto": "https", "cloudfront-is-desktop-viewer": "true", "cloudfront-is-mobile-viewer": "false", "cloudfront-is-smarttv-viewer": "false", "cloudfront-is-tablet-viewer": "false", "cloudfront-viewer-country": "US", "host": "jlmq3ygdu1.execute-api.us-west-1.amazonaws.com", "user-agent": "HTTPie/0.9.8", "via": "1.1 f1a40337a32137e1c23ceffead6a50d5.cloudfront.net (CloudFront)", "x-amz-cf-id": "Bt_hdRpgoK_u8oGdWSuYLo42AEdGxKYmJEnJjOeLbZ7og4P1kyMFZw==", "x-amzn-trace-id": 

2019-01-26 00:37:56.009000 a8ddcb s3 - ERROR - Internal Error for <function s3objects at 0x7f6689386730>
2019-01-26 00:37:56.009000 a8ddcb File "/var/task/app.py", line 35, in s3objects
2019-01-26 00:39:25.506000 882ee8 s3objects: SO FAR SO GOOD -----------------------------------------------------------------
2019-01-26 00:39:25.506000 882ee8 s3objects: <{'query_params': None, 'headers': {'accept': '*/*', 'accept-encoding': 'gzip, deflate', 'cloudfront-forwarded-proto': 'https', 'cloudfront-is-desktop-viewer': 'true', 'cloudfront-is-mobile-viewer': 'false', 'cloudfront-is-smarttv-viewer': 'false', 'cloudfront-is-tablet-viewer': 'false', 'cloudfront-viewer-country': 'US', 'host': 'jlmq3ygdu1.execute-api.us-west-1.amazonaws.com', 'user-agent': 'HTTPie/0.9.8', 'via': '1.1 24b0e5a3429d07ef12381da50e07f70f.cloudfront.net (CloudFront)', 'x-amz-cf-id': 'jJFLYx3gAXNkB8v5_zW3OgO7IYkbvDBktrUD8hZSt2qp20QVMpXIew==', 'x-amzn-trace-id': 'Root=1-5c4bac3c-2d30962cbacf1490cf4231f6', 'x-forwarded-for':

2019-01-26 01:03:44.869000 189dfc GET s3://mjbright-uploads/test
2019-01-26 01:04:01.431000 189dfc s3text: ----------------------------------------------------------------- <{'query_params': None, 'headers': {'accept': '*/*', 'accept-encoding': 'gzip, deflate', 'cloudfront-forwarded-proto': 'https', 'cloudfront-is-desktop-viewer': 'true', 'cloudfront-is-mobile-viewer': 'false', 'cloudfront-is-smarttv-viewer': 'false', 'cloudfront-is-tablet-viewer': 'false', 'cloudfront-viewer-country': 'US', 'host': 'jlmq3ygdu1.execute-api.us-west-1.amazonaws.com', 'user-agent': 'HTTPie/0.9.8', 'via': '1.1 49c80a47c1441dd194a8337982f1cd7e.cloudfront.net (CloudFront)', 'x-amz-cf-id': 'THP5eRWQ4cEezLBbXU-2vT4VzPDF4-mCFcMy6JMGOQTUd367pcvL4w==', 'x-amzn-trace-id': 'Root=1-5c4bb201-b56313b28f6fdc1ce3892232', 'x-forwarded-for': '54.67.51.193, 54.239.134.14', 'x-forwarded-port': '443', 'x-forwarded-proto': 'https'}, 'uri_params': {'key': 'test'}, 'method': 'GET', 'context': {'resourceId': 'ezap0f', 'resourceP

2019-01-26 01:04:02.449000 189dfc GET s3://mjbright-uploads/test
2019-01-26 01:04:02.528000 189dfc s3 - ERROR - Internal Error for <function s3json at 0x7f3be49ebd90>
2019-01-26 01:04:02.528000 189dfc File "/var/task/app.py", line 59, in s3json


In [112]:
echo hello world | http PUT :8000/text/test

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
HTTP/1.1 400 Bad Request
Content-Length: 74
Content-Type: application/json
Date: Sat, 26 Jan 2019 01:05:44 GMT
Server: BaseHTTP/0.6 Python/3.6.7

{
    "Code": "BadRequestError",
    "Message": "BadRequestError: Error Parsing JSON"
}



In [111]:
echo hello world | http PUT https://jlmq3ygdu1.execute-api.us-west-1.amazonaws.com/api/text/test

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
HTTP/1.1 400 Bad Request
Connection: keep-alive
Content-Length: 74
Content-Type: application/json
Date: Sat, 26 Jan 2019 01:05:14 GMT
Via: 1.1 8008015354a3ca72f56c382a1d1cfe9f.cloudfront.net (CloudFront)
X-Amz-Cf-Id: hJ_mqOfA_pDSF8wBR5xVPFHwYsiTK9WKOfJ0SEPPIXQljIw__LA52w==
X-Amzn-Trace-Id: Root=1-5c4bb24a-c4346c80ef290c006c5cab40;Sampled=0
X-Cache: Error from cloudfront
x-amz-apigw-id: UFjLoHZfyK4FlIg=
x-amzn-RequestId: 700050be-2106-11e9-9afc-0db0ce26a5a3

{
    "Code": "BadRequestError",
    "Message": "BadRequestError: Error Parsing JSON"
}



In [28]:
aws s3 help

S3()                                                                      S3()



NAME
       s3 -

DESCRIPTION
       This  section  explains  prominent concepts and notations in the set of
       high-level S3 commands provided.

   Path Argument Type
       Whenever using a command, at least one path argument must be specified.
       There are two types of path arguments: LocalPath and S3Uri.

       LocalPath: represents the path of a local file or directory.  It can be
       written as an absolute path or relative path.

       S3Uri: represents the location of a S3 object, prefix, or bucket.  This
       must  be  written in the form s3://mybucket/mykey where mybucket is the
       specified S3 bucket, mykey is the specified S3 key.  The path  argument
       must  begin with s3:// in order to denote that the path argument refers
       to a S3 object. Note that prefixes are separated  by  forward  slashes.
       For  example, if the S3 object myobject had the prefix myprefix,

In [21]:
aws s3 ls s3://mjbright-uploads/objects/

2019-01-26 00:19:55        900 test


In [65]:
aws s3 cp app.py s3://mjbright-uploads/test

upload: ./app.py to s3://mjbright-uploads/test                 


In [200]:
chalice logs --include-lambda-messages

In [198]:
aws lambda list-functions

{
    "Functions": [
        {
            "FunctionName": "chalice-app-dev-handler",
            "FunctionArn": "arn:aws:lambda:us-west-1:568285458700:function:chalice-app-dev-handler",
            "Runtime": "python3.6",
            "Role": "arn:aws:iam::568285458700:role/chalice-app-dev",
            "Handler": "app.handler",
            "CodeSize": 6116621,
            "Description": "",
            "Timeout": 60,
            "MemorySize": 128,
            "LastModified": "2019-01-26T02:26:42.254+0000",
            "CodeSha256": "WrtJGS2Xp8x7h2B/J869JIw9a7prGX20yCQXLKJzymo=",
            "Version": "$LATEST",
            "VpcConfig": {
                "SubnetIds": [],
                "SecurityGroupIds": [],
                "VpcId": ""
            },
            "TracingConfig": {
                "Mode": "PassThrough"
            },
            "RevisionId": "82800b08-5f85-4e50-b501-dfe42805c38e"
        }
    ]
}


# S3 Events

In [189]:
pwd
cat >app.py <<EOF

from chalice import Chalice

app = Chalice(app_name="s3events")
app.debug = True


# Whenever an object is uploaded to 'mybucket'
# this lambda function will be invoked.

@app.on_s3_event(bucket='mjbright-uploads')
def handler(event):
    print("S3Event: Object uploaded for bucket: %s, key: %s"
          % (event.bucket, event.key))

EOF

/home/user1/src/git/ServerlessLabs/ServerlessWorkshop/AWS-S3-Lambda/chalice-app


In [190]:
chalice deploy

/usr/share/python-wheels/requests-2.18.4-py2.py3-none-any.whl/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
Creating deployment package.
Updating policy for IAM role: chalice-app-dev
Updating lambda function: chalice-app-dev-handler
Configuring S3 events in bucket mjbright-uploads to function chalice-app-dev-handler
Resources deployed:
  - Lambda ARN: arn:aws:lambda:us-west-1:568285458700:function:chalice-app-dev-handler


In [191]:
pwd
ls -al
aws s3 cp .gitignore s3://mjbright-uploads/g2314

/home/user1/src/git/ServerlessLabs/ServerlessWorkshop/AWS-S3-Lambda/chalice-app
total 40
drwxrwxr-x  6 user1 user1 4096 Jan 26 00:41 .
drwxrwxr-x 10 user1 user1 4096 Jan 26 02:21 ..
drwxrwxr-x  4 user1 user1 4096 Jan 25 00:00 .chalice
-rw-rw-r--  1 user1 user1   37 Jan 24 22:33 .gitignore
drwxrwxr-x  3 user1 user1 4096 Jan 25 01:32 0
drwxrwxr-x  2 user1 user1 4096 Jan 26 02:26 __pycache__
-rw-rw-r--  1 user1 user1  337 Jan 26 02:26 app.py
drwxrwxr-x  4 user1 user1 4096 Jan 26 00:11 chalice-app
-rw-rw-r--  1 user1 user1   28 Jan 25 02:41 requirements.txt
-rw-rw-r--  1 user1 user1   13 Jan 26 01:51 test
upload: ./.gitignore to s3://mjbright-uploads/g2314               


In [ ]:
chalice logs --include-lambda-messages

In [232]:
cat > app.py <<EOF

from chalice import Chalice
from chalice import NotFoundError

app = Chalice(app_name='s3')

@app.route('/')
def index():
    return {'hello': 'world'}

import json
import boto3
from botocore.exceptions import ClientError

S3 = boto3.client('s3', region_name='us-west-2')
BUCKET = 'mjbright-uploads'

@app.route('/objects/{key}', methods=['GET', 'PUT'])
def s3objects(key):
    request = app.current_request
    if request.method == 'PUT':
        S3.put_object(Bucket=BUCKET, Key=key,
                      Body=json.dumps(request.json_body))
    elif request.method == 'GET':
        try:
            response = S3.get_object(Bucket=BUCKET, Key=key)
            return json.loads(response['Body'].read())
        except ClientError as e:
            raise NotFoundError(key)

EOF

In [232]:
aws s3 rb --force s3://mjbright-uploads/
aws s3 mb s3://mjbright-uploads/

Just for local testing - cannot work as cannot access S3 credentials.

In [232]:
aws s3 ls s3://mjbright-uploads
echo '["testuser", {"fullname": "My full name"}]' | http PUT :8000/objects/testfile
aws s3 ls s3://mjbright-uploads

In [232]:
chalice deploy

In [232]:
echo '["testuser", {"fullname": "My full name"}]' | http PUT $(chalice url)/objects/testfile

In [232]:
aws s3 ls s3://mjbright-uploads/

In [232]:
aws s3 cp s3://mjbright-uploads/testfile -

In [232]:
BUCKET_NAME='mjbright-uploads'

cat > app.py <<EOF
from chalice import Chalice
#import sys

app = Chalice(app_name="helloworld")

# Whenever an object is uploaded to 'mybucket'
# this lambda function will be invoked.

@app.on_s3_event(bucket='$BUCKET_NAME')
def handler(event):
    # BUT WHERE DOES THIS OUTPUT GO?
    print("Object uploaded for bucket: %s, key: %s"
          % (event.bucket, event.key))
    #sys.exit(0)
EOF

In [232]:
cat app.py

Deploy a local chalice server in another window using ```chalice local```, then upload a file to our bucket

In [232]:
cat > requirements.txt <<EOF
boto3==1.3.1
EOF

In [232]:
aws lambda delete-function --function-name chalice-app-dev-handler

In [232]:
cat app.py


In [232]:
cat requirements.txt

In [232]:
aws lambda list-functions

In [232]:
# Only works when deployed?
chalice deploy

In [232]:
aws s3 cp /etc/hosts s3://mjbright-uploads/1
aws s3 ls  s3://mjbright-uploads

# More Reading

You can find more details about S3 website hosting here: https://docs.aws.amazon.com/AmazonS3/latest/dev/WebsiteHosting.html

An article describing the use of S3 for static website hosting including use of https, DNS routing
https://medium.freecodecamp.org/how-to-host-a-static-website-with-s3-cloudfront-and-route53-7cbb11d4aeea

# Cleanup

Note that we can use the ```aws s3 rm``` command to remove files from the bucket and ```aws s3 rb``` command to remove a bucket.

In [ ]:
aws s3 rm s3://mjbright-static-site --recursive
aws s3 rb s3://mjbright-static-site

It's also possible to remoce the bucket directly using the ```--force``` option:
    ```aws s3 rb --force s3://mjbright-static-site```